In [1]:
# Importing the libraries
import torch
#from torchviz import make_dot
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import NN_classes

In [2]:
executions = ["Network_Full_Generation_Full","Network_Line_Out_Node_1_Node_2_cac1"]
sc = "sc01"
period = "2030"
train_size = 0.8

In [14]:
dfs_in = dict()
dfs_out = dict()
for execution in executions:
    #Read the data from desired execution
    df_in_e = pd.read_csv(f"Data/input_f_{sc}_{execution}_{period}.csv",header=[0,1])
    df_out_e = pd.read_csv(f"Data/output_f_{sc}_{execution}_{period}.csv",header=[0,1])

    #Drop the first row because its useless 
    df_in_e = df_in_e.drop([0])
    df_out_e = df_out_e.drop([0])
    print(f"Data/input_f_{sc}_{execution}_{period}.csv")
    

    #Focus for now only on the real part of the input data: 
    df_in_e_r = df_in_e["Value_R"]
    #df_in_e_i = df_in_e["Value_I"].loc[:, (df_in_e["Value_I"] != 0).any(axis=0)]
    df_in_e_i = df_in_e["Value_I"].drop(["Node_1", "Node_2","Node_3","SolarPV_1","WindFarm_1"],axis=1)
    
    print(len(df_in_e_r.columns)+ len(df_in_e_i.columns))
    
    df_in_e_c = pd.concat([df_in_e_r,df_in_e_i],axis=1)
    df_out_e = df_out_e["Value"]
    dfs_in[execution] = df_in_e_c
    dfs_out[execution] = df_out_e


Data/input_f_sc01_Network_Full_Generation_Full_2030.csv
23
Data/input_f_sc01_Network_Line_Out_Node_1_Node_2_cac1_2030.csv
23


In [15]:
ts_in  = dict()
ts_out = dict()

ts_in["train"] = dict()
ts_in["test"]  = dict()
ts_out["train"]= dict()
ts_out["test"] = dict()


test_size = 0.3

for execution in executions: 
    #Convert input dataframes numpy arrays sum the columns of the output: 
    np_in = dfs_in[execution].to_numpy()
    #We don't normalize the separate runs, but will do it afterward, all together
    #np_in = np_in/np.abs(np_in.max(axis=0))
    #np_in = np.nan_to_num(np_in,nan = 0)
    np_out = dfs_out[execution].to_numpy().sum(axis=1)

    #Convert to torch tensors
    t_in = torch.from_numpy(np_in)
    t_out = torch.from_numpy(np_out)

    #And split into train and test set:
    ts_in["train"][execution],ts_in["test"][execution],ts_out["train"][execution],ts_out["test"][execution]= train_test_split(t_in,t_out,test_size=test_size,shuffle=False)
#     train_size = 0.8
#     nb_train = int(train_size*len(t_out))
#     tr_in,te_in = t_in[:nb_train],t_in[nb_train:]
#     tr_out,te_out = t_out[:nb_train],t_out[nb_train:]
    
    


In [16]:
#concatenate all the training and testing sets to a single tensor, and normalize: 
first = True
for execution in executions:
    if first: 
        tr_in = ts_in["train"][execution]
        tr_out = ts_out["train"][execution]
        te_in = ts_in["test"][execution]
        te_out = ts_out["test"][execution]
    else: 
        torch.cat(tr_in,ts_in["train"][execution])
        torch.cat(tr_out,ts_out["train"][execution])
        torch.cat(te_in,ts_in["test"][execution])
        torch.cat(te_out,ts_out["test"][execution])
        
        
    

In [36]:
tr_in.abs().max(dim = 0).values

tensor([0.0335, 0.9794, 0.0000, 0.9794, 0.0606, 0.0000, 0.0000, 0.0000, 0.1842,
        0.9794, 0.0000, 0.9794, 0.0468, 0.1000, 0.1420, 0.0000, 0.1420, 0.0000,
        0.0000, 0.0000, 0.1420, 0.0000, 0.1420], dtype=torch.float64)